# Select:


* Pairs with high agreement/few contradictions
* Pairs with low agreement/few contradictions (if possible)
* Pairs with high agrement/many contradictions
* Pairs with low agreement/many contradictions

Try to spread over property types


# Annotate:

* New run
* Define new experiment group somehow

# Ideas:

* Low agreement, low contradiction rate: knowledge issue (some people know, some don't)
* Low agreement, high contradiction rate: ambiguity



# Implementation plan:

* sort according to agreement
* sort according to contradiction rate
* get top pairs (top 5 or so)
* get intersections:
    * most problematic: low agreement - high contradiction
    * problematic: low agreement - low contradiction
    * interesting: high agreement - high contradiction (multiple interpretations?)
    * clear: high agreement - low contradiction

In [1]:
from load_data import load_experiment_data
from calculate_iaa import get_agreement
from utils_analysis import load_contradiction_pairs
from utils_analysis import collect_contradictions
from utils_analysis import sort_by_key
from utils_analysis import get_annotation_ids
from utils_analysis import load_analysis

from collections import Counter, defaultdict
import pandas as pd
from scipy.stats import spearmanr


def get_agreement_by_property(data_dict_list):

    agreement_prop_dict = dict()
    data_by_pair = sort_by_key(data_dict_list, ['property', 'concept'])
    for pair, dl_prop in data_by_pair.items():
        agreement_prop_dict[pair] = get_agreement(dl_prop, v=False, disable_kappa=True)
    return agreement_prop_dict


def get_pairs_by_day(data_dict_list):
    data_by_date = sort_by_key(data_dict_list, ['timestamp'])
    day_by_pair = dict()
    for d, data in data_by_date.items():
        day = d.split(' ')[0]
        pairs = sort_by_key(data, ['property', 'concept']).keys()
        for pair in pairs:
            day_by_pair[pair] = day
    return day_by_pair


def get_agreement_contradiction_data(run, group, batch, n_q):
    data_dict_list = load_experiment_data(run, group, n_q, batch, remove_not_val = True)
    print(run, group, batch, n_q)
    #data_by_pair = sort_by_key(data_dict_list, ['property', 'concept'])
    analysis_type = 'pairs'
    df = load_analysis(analysis_type, run, group, batch)
    pair_dicts = df.to_dict('records') 
    pair_dicts_by_pair = sort_by_key(pair_dicts, ['pair'])
    ag_pair_dict = get_agreement_by_property(data_dict_list)
    # get dates
    day_by_pair = get_pairs_by_day(data_dict_list)
    print(day_by_pair['round-fruit'])
    agreement_dict = Counter()
    for pair, ag in ag_pair_dict.items():
        agreement_dict[pair] =  ag['Krippendorff']

    ag_cont_dicts = []
    for pair, ag in agreement_dict.most_common():
        date = day_by_pair[pair]
        if 'test' not in pair and 'check' not in pair: 
            d = pair_dicts_by_pair[pair]
            if len(d) == 1:
                new_dict = dict()
                d = d[0]
                #print(d.keys())
                cont = d['contradiction_poss_contradiction_ratio']
                new_dict['pair'] = pair
                new_dict['agreement'] = ag
                new_dict['contradiction_rate'] = cont
                new_dict['date'] = date
                ag_cont_dicts.append(new_dict)

            else:
                print('unexpected length:', len(d), 'for pair', pair)
        else:
            print('test pair:', pair)
        #d = data_by_pair[pair]
    df_ag_cont = pd.DataFrame(ag_cont_dicts)
    return df_ag_cont


In [2]:
run = "*"
group = 'experiment*'
batch = '*'
n_q = '*'

df_cont_ag = get_agreement_contradiction_data(run, group, batch, n_q)


Discarded 655.0 annotations.
* experiment* * *
23-Mar-2020
test pair: _test4-_test
test pair: _check4-_check4
test pair: _test2-_test2
test pair: _check2-_check2
test pair: _test1-_test1
test pair: _test3-_test
test pair: _check3-_check3
test pair: _check1-_check1
test pair: _test4-_test4
test pair: _test3-_test3


In [ ]:
#data_dict_list = load_experiment_data(run, group, n_q, batch, remove_not_val = True)
#data_by_day = get_pairs_by_day(data_dict_list)

In [3]:
# High agreement

df_ag_sorted = df_cont_ag.sort_values('agreement', axis = 0, ascending=False, inplace=False)
top_ag = df_ag_sorted[:30]
print(f'Total number of pairs: {len(df_ag_sorted)}')
top_ag

Total number of pairs: 1935


,agreement,contradiction_rate,date,pair
0,1.000000,0.000000,03-Jun-2020,made_of_wood-pen
1,1.000000,0.000000,19-May-2020,yellow-citrus
2,0.960384,0.000000,28-May-2020,sweet-cookie
3,0.929487,0.000000,04-Jun-2020,sweet-berry
4,0.921667,0.000000,02-Jun-2020,wheels-lorry
5,0.878654,0.000000,02-Jun-2020,dangerous-poison
6,0.857143,0.000000,25-May-2020,wheels-motorcar
7,0.855885,0.000000,04-Jun-2020,sweet-peach
8,0.850400,0.000000,05-Jun-2020,sweet-fruit
9,0.850160,0.033333,26-May-2020,hot-cooker


In [57]:
# Low agreement

df_ag_sorted = df_cont_ag.sort_values('agreement', axis = 0, ascending=False, inplace=False)
bottom_ag = df_ag_sorted[-30:-1]
bottom_ag

,agreement,contradiction_rate,date,pair
1905,-0.049601,0.186667,07-May-2020,lay_eggs-hobby
1906,-0.049689,0.060000,23-Apr-2020,lay_eggs-whiteface
1907,-0.050434,0.000000,13-May-2020,dangerous-freebooter
1908,-0.051556,0.000000,08-Jun-2020,yellow-shiitake
1909,-0.051610,0.000000,06-May-2020,yellow-buttercup
1910,-0.052126,0.000000,20-May-2020,square-book
1911,-0.052632,0.000000,26-May-2020,yellow-potato
1912,-0.052856,0.000000,15-May-2020,wheels-sleigh
1913,-0.053467,0.000000,27-May-2020,made_of_wood-trumpet
1914,-0.057540,0.000000,08-Jun-2020,yellow-dogwood


In [4]:
# middle

df_ag_sorted = df_cont_ag.sort_values('agreement', axis = 0, ascending=False, inplace=False)
bottom_ag = df_ag_sorted[-720:-730+40]
bottom_ag

,agreement,contradiction_rate,date,pair
1215,0.186275,0.033333,26-May-2020,wheels-runabout
1216,0.186275,0.066667,28-Apr-2020,black-pea
1217,0.185768,0.075000,27-Mar-2020,round-breadcrumb
1218,0.185185,0.066667,26-May-2020,dangerous-acaridae
1219,0.184891,0.000000,19-May-2020,yellow-sandpaper
1220,0.183711,0.000000,04-Jun-2020,black-bra
1221,0.183612,0.033333,19-May-2020,sweet-parsnip
1222,0.182857,0.053333,15-May-2020,fly-towhee
1223,0.182857,0.000000,27-May-2020,square-hallway
1224,0.182857,0.033333,25-May-2020,black-rhino


In [62]:
def pairs_to_file(name, pairs):
    path = f'../analyses/{name}.txt'
    with open(path, 'w') as outfile:
        for p in pairs:
            outfile.write(p+'\n')
    print(f'Written pairs to {path}')
    

# Selecting data 

* from high, low and medium agreement
* pairs workers have commented on


## Worker comments:

* Dangerous - guttering
* fly-trumpeter
* lay_eggs - hackney
* Yellow - Corn on the cob 
* Made from wood - brush
* Hot - chutney
* Has_wheels - equipment 
* sweet - desert (mistaken for dessert)

In [64]:
# Selection for a trial round:

pair_dict = dict()
all_pairs = []

pair_dict['expert_inspection1'] = ['fly-arrow', 'roll-pin', 'yellow-buttercup',
                                   'hot-vineigrette', 'roll-tire']
pair_dict['expert_inspection2'] = ['roll-shovel', 'dangerous-freebooter',
                                   'yellow-pineapple', 'red-carrot', 'red-wine']
pair_dict['expert_inspection3'] = ['square-recliner', 'yellow-leopard', 
                                   'round-pen', 'roll-washer', 'lay_eggs-plaice']
#pair_dict['expert_inspection4'] = ['dangerous-guttering','fly-trumpeter', 'lay_eggs-hackney', 
                                  # 'yellow-corn_on_the_cob','made_from_wood-brush', 'hot-chutney']


pair_dict['expert_inspection4'] = ['dangerous-acaridae','fly-stock', 'wheels-cruiser', 
                                   'black-rhino','made_from_wood-brush', 'made_of_wood-pen']
all_pairs = []
for name, pairs in pair_dict.items():
    if any([p in all_pairs for p in pairs]):
        print(f'Attention! Duplicate pair in {name}')
    else:
        all_pairs.extend(pairs)
        pairs_to_file(name, pairs)
        

Written pairs to ../analyses/expert_inspection1.txt
Written pairs to ../analyses/expert_inspection2.txt
Written pairs to ../analyses/expert_inspection3.txt
Written pairs to ../analyses/expert_inspection4.txt
